# Scraper für WGgesucht - München

Ausgangslink: https://www.wg-gesucht.de/wg-zimmer-in-Muenchen.90.0.1.0.html

Bibliotheken installieren


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

First part of the code gets all Announces in the specified pages (10) by URL and saves it to a list

In [3]:
def api_call(url):
  payload = {'api_key': '33bdafd2e116ede406243b5c802b1145', 'url': url}
  try:
    response = requests.get('http://api.scraperapi.com', params=payload)
  except ConnectionError:
    print("Connection Error")
    exit(0)
  
  return response

In [4]:
  page_amount = 50

url_first_part = 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen.90.0.1.'
url_second_part = '.html'                                                                       #complete url => first part + page_number + second part

total_link_list = []

for page_number in range(1, page_amount):

    response = api_call(url_first_part + str(page_number) + url_second_part)
    soup = BeautifulSoup(response.content, "html.parser")

    relevant_page_content = soup.find("div", attrs={'class': 'col-xs-12 col-md-8 col-md-push-4'})
    links = relevant_page_content.find_all("a", attrs={'class': 'detailansicht'})

    even = True
    for link in links[1:]:
        intern_link = link.get("href")
        if even:
            if "sort_column" not in intern_link:
                total_link_list.append("https://www.wg-gesucht.de/" + intern_link)
            even = False
        else:
            even = True

print(total_link_list)


['https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Schwabing-Freimann.11022480.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Maxvorstadt.4604402.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Moosach.10795254.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Schwabing.9873207.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Obergiesing-Fasangarten.10995347.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Schwabing-Freimann.9330148.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Aubing-Lochhausen-Langwied.10461331.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Untergiesing-Harlaching.10209578.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Giesing.10278173.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Altstadt-Lehel.9942421.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen.10653977.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Allach-Untermenzing.7885803.html', 'https://www.wg-gesucht.de/wg-zimmer-i

In [5]:
def get_page_content(page_content, Url):
    try:
        title = page_content.find("h1", attrs={'class': 'headline headline-detailed-view-title'}).text.strip()             # get title String
        
        size = page_content.find_all("b", attrs={'class': 'key_fact_value'})[0].text.strip()             # get size String

        price_total = page_content.find_all("b", attrs={'class': 'key_fact_value'})[1].text.strip()             # get price_total String

        price = page_content.find_all("span", attrs={'class': 'section_panel_value'})[0].text.strip()             # get price String

        extra_costs = page_content.find_all("span", attrs={'class': 'section_panel_value'})[1].text.strip()             # get extra_costs String

        other_costs = page_content.find_all("span", attrs={'class': 'section_panel_value'})[2].text.strip()             # get other_costs String

        deposit = page_content.find_all("span", attrs={'class': 'section_panel_value'})[3].text.strip()             # get deposit String

        redemption_agreement = page_content.find_all("span", attrs={'class': 'section_panel_value'})[4].text.strip()             # get redemption_agreement String

        date_free = page_content.find_all("span", attrs={'class': 'section_panel_value'})[5].text.strip()             # get date_free String

        date_free_till= "n.a."
        if len(page_content.find_all("span", attrs={'class': 'section_panel_value'})) > 6:
            date_free_till = page_content.find_all("span", attrs={'class': 'section_panel_value'})[6].text.strip()             # get date_free_till String
               
        online_since =  page_content.find("b", attrs={'class': 'noprint'}).text.strip()             # get online_since String
        
        adress = page_content.find_all("span", attrs={'class': 'section_panel_detail'})[5].text.strip()             # get adress String

        details = ""
        for detail in page_content.find_all("ul", attrs={'class': 'pl15'})[0].find_all("span", attrs={'class': 'section_panel_detail'}):              #get Details String
            details += detail.text.strip() + '\n'

        looking_for = ""
        for looking_for_element in page_content.find_all("ul", attrs={'class': 'pl15'})[1].find_all("span", attrs={'class': 'section_panel_detail'}):              #get looking_for String
            looking_for += looking_for_element.text.strip() + '\n'

        amount_of_icons = "n.a. "
        if len(page_content.find_all("div", attrs={'class': 'utility_icons'})) > 0:
            amount_of_icons = len(page_content.find_all("div", attrs={'class': 'utility_icons'})[0].find_all("div", attrs={'class': 'text-center'}))         # get amount_of_icons String
        
        amount_of_icons_needed = "n.a."
        if len(page_content.find_all("div", attrs={'class': 'utility_icons'})) > 1:
            counter = 0
            counter += len(page_content.find_all("div", attrs={'class': 'utility_icons'})[1].find_all("div", attrs={'class': 'text-center'}))         # get amount_of_icons String
            counter += len(page_content.find_all("div", attrs={'class': 'utility_icons'})[1].find_all("a", attrs={'class': 'text-center campaign_click'}))
            amount_of_icons_needed = counter

        description_room_div = page_content.find("div", attrs={'id': 'freitext_0'})
        description_room = 'n.a.'
        if description_room_div:
            description_room = description_room_div.text.strip()
                    
        description_location_div = page_content.find("div", attrs={'id': 'freitext_1'})
        description_location = 'n.a.'
        if description_location_div:
            description_location = description_location_div.text.strip()
      
        # Check and get description_life if it exists
        description_life_div = page_content.find("div", attrs={'id': 'freitext_2'})
        description_life = 'n.a.'
        if description_life_div:
            description_life = description_life_div.text.strip()
       
        # Check and get description_other if it exists
        description_other_div = page_content.find("div", attrs={'id': 'freitext_3'})
        description_other = 'n.a.'
        if description_other_div:
            description_other = description_other_div.text.strip()
                    

        return {'Url': Url, 'Title': title, 'Size': size, 'PriceTotal': price_total, 'Price': price, 'ExtraCosts': extra_costs, 'OtherCosts': other_costs, 'Deposit': deposit, 'RedemptionAgreement': redemption_agreement, 'DateFree': date_free, 'DateFreeTill': date_free_till, 'OnlineSince': online_since, 'Adress': adress, 'Details': details, 'LookingFor': looking_for, 'AmountofIcons': amount_of_icons, 'AmountofIconsNeeded': amount_of_icons_needed, 'DescriptionRoom': description_room, 'DescriptionLocation': description_location, 'DescriptionLife': description_life, 'description_other': description_other}
    except (AttributeError, IndexError) as e:
        return "error"


In [6]:
main_dict = {'Url': [], 'Title': [], 'Size': [], 'PriceTotal': [], 'Price': [], 'ExtraCosts': [], 'OtherCosts': [], 'Deposit': [], 'RedemptionAgreement': [], 'DateFree': [], 'DateFreeTill': [], 'OnlineSince': [], 'Adress': [], 'Details': [], 'LookingFor': [], 'AmountofIcons': [], 'AmountofIconsNeeded': [], 'DescriptionRoom': [], 'DescriptionLocation': [], 'DescriptionLife': [], 'description_other': []}

for link in total_link_list:
    
    response = api_call(link)
    soup = BeautifulSoup(response.content, "html.parser")
    temp_dict = get_page_content(soup, link)
    if temp_dict == "error":
        continue    
    for key, value in temp_dict.items():
        if key in main_dict:
            main_dict[key].append(value) 
        else:
            main_dict[key] = [value]  

    
result_dataframe = pd.DataFrame(main_dict)

print(result_dataframe.size)
display(result_dataframe)

# Save DataFrame to CSV without the index
result_dataframe.to_csv('wggesucht_scraped_24_05.csv', index=False)   


23835


,Url,Title,Size,PriceTotal,Price,ExtraCosts,OtherCosts,Deposit,RedemptionAgreement,DateFree,...,OnlineSince,Adress,Details,LookingFor,AmountofIcons,AmountofIconsNeeded,DescriptionRoom,DescriptionLocation,DescriptionLife,description_other
0,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Room in a WG of 4 close to Freimann U6,12m²,670€,670€,n.a.,n.a.,n.a.,n.a.,01.07.2024,...,32 Minuten,Freisinger Landstr. 18\n ...,12m²\n \n ...,Geschlecht egal\n ...,3,n.a.,"The room is furnished with desk, bed and wardr...",Very close to the Freimann stop (U6). Buses 18...,You will be sharing the apartment with me (Ita...,"The kitchen, which is a separate room in the a..."
1,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Beste Lage - Zimmer frei bei Studentenverbindung,20m²,350€,350€,n.a.,0€,600€,0€,24.05.2024,...,6 Minuten,Königinstr. 20\n ...,20m²\n \n ...,Mann\n ...,12,n.a.,Du wirst demnächst Dein Studium an der LMU/TU ...,Top Lage direkt am Englischen Garten - Zweimal...,"Wir sind ein ""Corps"", eine Art von Studentenv...","Da wir besonderen Wert darauf legen, dass wir ..."
2,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP LAGE - U3 Nahe Olympia-Einkaufszentrum,11m²,790€,740€,50€,0€,2200€,0€,24.05.2024,...,37 Minuten,Seydlitzstraße 63\n ...,11m²\n \n ...,Geschlecht egal\n,8,4,Gemütliches WG Zimmer voll möbliert mit Bett m...,"Sehr gute Lage, Bus-und U-Bahn-Haltestellen na...",Gemischte Wohngemeinschaft mit Studenten und B...,Verfügbar ab sofort.
3,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP UNI LOCATION near U3/U6: Münchner Freiheit,14m²,970€,911€,59€,0€,2673€,0€,01.08.2024,...,37 Minuten,Germaniastraße 3\n ...,14m²\n \n ...,Geschlecht egal\n,7,3,"Schlafzimmer ist voll Möbliert (Bett, Schrank,...",Die Wohnung befindet sich in einer spitzen Lag...,4 junge Männer Anfang 20 Jahre. \n\r\n Gemeins...,Bitte kontaktieren Sie uns über WG-Gesucht. \r...
4,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,+++ WG-Zimmer in renovierter 4er WG verfügbar!...,14m²,800€,715€,85€,0€,2145€,0€,01.07.2024,...,39 Minuten,Schlierseestraße 14\n ...,14m²\n \n ...,Geschlecht egal\n,12,5,Wunderschönes Zimmer in 4er WG ab dem 01.07.20...,Die Schlierseestraße 14 liegt in einer attrakt...,Willkommen in unserer charmanten WG in München...,Link zum Wohnungs-Bewerbungsformular (Aplicati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Wohngemeinschaft in München,108m²,1220€,1220€,n.a.,n.a.,1830€,n.a.,05.08.2024,...,17.05.2024,Tumblingerstraße Glockenbachviertel 17\n ...,108m²\n \n ...,Geschlecht egal\n,3,n.a.,Wohnen Sie nur wenige Schritte von der Isar en...,munich,n.a.,**Bitte beachten: PAARE** sind nur in Schlafzi...
1131,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,n.a.,1325€,1325€,n.a.,n.a.,1000€,n.a.,24.05.2024,...,17.05.2024,Heßstraße 74\n ...,Zimmer in 2er WG\nWohnungsgröße: 1m²\n2er WG\n...,Geschlecht egal\n,4,n.a.,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,munich,n.a.,**Dieser Vermieter akzeptiert keine Studenten!...
1132,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Privatzimmer in Laim...,12m²,1020€,1020€,n.a.,n.a.,n.a.,n.a.,19.08.2024,...,1 Tag,Tübinger Straße 3c\n ...,12m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-896965
1133,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Private Room in Unte...,14m²,960€,960€,n.a.,n.a.,n.a.,n.a.,05.08.2024,...,1 Tag,Grünwalderstraße 119\n ...,14m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-380836
